In [ ]:
import json
import random
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
# Unduh resource NLTK
# nltk.download('punkt_tab')
# nltk.download('stopwords')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\grubg\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\grubg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Load dataset
with open("./Assets/e-commerce.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Preprocessing helper
stemmer = PorterStemmer()
stop_words = set(stopwords.words('english'))

def preprocess(text):
    tokens = nltk.word_tokenize(text.lower())
    tokens = [stemmer.stem(t) for t in tokens if t.isalpha() and t not in stop_words]
    return " ".join(tokens)

# Siapkan data training
X = []
y = []
responses = {}
for intent in data["intents"]:
    tag = intent["tag"]
    for pattern in intent["patterns"]:
        X.append(preprocess(pattern))
        y.append(tag)
    responses[tag] = intent["responses"]

print(type(X))
# Cek Data
print(len(X), len(y))
for i in range(16):
    print(X[i], '\t', y[i])

print('...')
for i in range(4000, 4016):
    print(X[i], '\t', y[i])
    
print('...')
for i in range(len(X)-1, len(X)-21, -1):
    print(X[i], '\t', y[i])

KeyError: 'intetns'

In [ ]:
# Buat pipeline: CountVectorizer + MNB
model = Pipeline([
    ("vectorizer", CountVectorizer()),
    ("classifier", MultinomialNB())
])

print(model)

In [ ]:
import numpy as np

# Train model
model.fit(X, y)

def chat():
    print("Chatbot siap! (Ketik 'keluar' untuk berhenti)\n")
    while True:
        user_input = input("Kamu: ")
        if user_input.lower() == "keluar":
            print("Bot: Sampai jumpa!")
            break

        preprocessed = preprocess(user_input)
        proba = model.predict_proba([preprocessed])[0]
        max_proba = np.max(proba)

        if max_proba < 0.2:
            print("Sorry I’m don’t understand")
        else:
            predicted_intent = model.classes_[np.argmax(proba)]
            response = random.choice(responses[predicted_intent])
            print("Bot:", response)

# Jalankan chatbot
if __name__ == "__main__":
    chat()


In [ ]:
import pickle
with open("model_chatbot.pkl", "wb") as model_file:
    pickle.dump(model, model_file)
